In [1]:
import torch

print("PyTorch version:", torch.__version__)

if torch.backends.mps.is_available():
    print("MPS (Apple Silicon GPU) available: True")
    print("Running on Apple GPU – Metal backend")
else:
    print("MPS (Apple Silicon GPU) available: False")
    print("Running on CPU – training will be slower")
    
device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")
print("Selected device:", device)
print("Hello, PyTorch on Apple Silicon!")

PyTorch version: 2.9.0
MPS (Apple Silicon GPU) available: True
Running on Apple GPU – Metal backend
Selected device: mps
Hello, PyTorch on Apple Silicon!


In [2]:
pip install --upgrade pip

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [3]:
# ============================================================================
# CELL 2: Install All Required Packages
# ============================================================================
# This will take 2-3 minutes - you'll see lots of output, that's normal!


# Install machine learning libraries
%pip install -q 'stable-baselines3[extra]'  # RL algorithms (quoted for zsh)
%pip install -q gymnasium                  # RL environment framework
%pip install -q scikit-learn              # Random Forest
%pip install -q pandas numpy              # Data manipulation
%pip install -q matplotlib seaborn        # Visualization
%pip install -q tqdm                      # Progress bars
%pip install -q requests gdown            # File downloading

print("✅ All packages installed successfully!")

import sys
print("\nVerifying installations:")
print(f"Python version: {sys.version.split()[0]}")

try:
    import stable_baselines3
    print(f"✓ stable-baselines3: {stable_baselines3.__version__}")
except:
    print("✗ stable-baselines3: NOT INSTALLED")

try:
    import gymnasium
    print(f"✓ gymnasium: {gymnasium.__version__}")
except:
    print("✗ gymnasium: NOT INSTALLED")

try:
    import sklearn
    print(f"✓ scikit-learn: {sklearn.__version__}")
except:
    print("✗ scikit-learn: NOT INSTALLED")

try:
    import pandas
    print(f"✓ pandas: {pandas.__version__}")
except:
    print("✗ pandas: NOT INSTALLED")

try:
    import matplotlib
    print(f"✓ matplotlib: {matplotlib.__version__}")
except:
    print("✗ matplotlib: NOT INSTALLED")

print("\n✅ Ready to proceed!")


Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
✅ All packages installed successfully!

Verifying installations:
Python version: 3.12.7
Note: you may need to restart the kernel to use updated packages.
✅ All packages installed successfully!

Verifying inst

Libraries Import

In [4]:
# ============================================================================
# CELL 2: Import All Libraries (RUN AFTER CELL 1)
# ============================================================================

# Standard libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm
import time
import warnings
warnings.filterwarnings('ignore')

# Machine Learning
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import (accuracy_score, precision_recall_fscore_support, 
                             confusion_matrix, classification_report)
from sklearn.model_selection import train_test_split

# Deep Learning & RL
import torch
import torch.nn as nn
import gymnasium as gym
from gymnasium import spaces
import stable_baselines3
from stable_baselines3 import PPO, SAC, TD3, A2C, DDPG, DQN
from stable_baselines3.common.env_checker import check_env

# Utilities
import os
import pickle
import json
from collections import defaultdict

print("✅ All libraries imported successfully!")
print(f"\nVersions:")
print(f"  NumPy: {np.__version__}")
print(f"  Pandas: {pd.__version__}")
print(f"  PyTorch: {torch.__version__}")
print(f"  Stable-Baselines3: {stable_baselines3.__version__}")
print(f"  Gymnasium: {gym.__version__}")


✅ All libraries imported successfully!

Versions:
  NumPy: 1.26.4
  Pandas: 2.2.2
  PyTorch: 2.9.0
  Stable-Baselines3: 2.7.0
  Gymnasium: 1.2.2


Kinematic Datasets

In [5]:
# ============================================================================
# CELL 4: Create Sample Kinematic Dataset (OPTIMIZED - FASTER)
# ============================================================================
# Reduced dataset size for faster training

def create_synthetic_kinematic_dataset(n_samples=1000, n_classes=10, n_features=100):
    """
    Create synthetic kinematic features similar to skeleton action data
    
    OPTIMIZED: Reduced from 5000 samples, 20 classes, 194 features
    
    Parameters:
    - n_samples: Number of video samples (reduced to 1000)
    - n_classes: Number of action classes (reduced to 10)
    - n_features: Feature dimension (reduced to 100)
    """
    print(f"Creating LIGHTWEIGHT dataset with {n_samples} samples...")
    
    np.random.seed(42)
    
    # Generate features with some structure (not pure random)
    X = []
    y = []
    
    for class_id in range(n_classes):
        # Each class has different feature distribution
        samples_per_class = n_samples // n_classes
        
        # Create class-specific mean and variance
        class_mean = np.random.randn(n_features) * 2 + class_id * 0.5
        class_std = np.random.rand(n_features) * 0.5 + 0.5
        
        # Generate samples
        class_samples = np.random.randn(samples_per_class, n_features) * class_std + class_mean
        
        X.append(class_samples)
        y.extend([class_id] * samples_per_class)
    
    X = np.vstack(X)
    y = np.array(y)
    
    # Shuffle
    indices = np.random.permutation(len(X))
    X = X[indices]
    y = y[indices]
    
    print(f"✅ Dataset created: {X.shape[0]} samples, {X.shape[1]} features, {n_classes} classes")
    
    return X, y

# Create SMALLER dataset for faster execution
X, y = create_synthetic_kinematic_dataset(n_samples=1000, n_classes=10, n_features=100)

print("\nDataset statistics:")
print(f"Feature matrix shape: {X.shape}")
print(f"Label array shape: {y.shape}")
print(f"Feature range: [{X.min():.2f}, {X.max():.2f}]")
print(f"Class distribution: {np.bincount(y)}")

# ============================================================================
# CELL 5: Split Data into Train and Test Sets
# ============================================================================

# Split: 70% train, 30% test
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=42, stratify=y
)

print("Data split completed:")
print(f"Training set: {X_train.shape[0]} samples")
print(f"Test set: {X_test.shape[0]} samples")
print(f"Train classes: {len(np.unique(y_train))}")
print(f"Test classes: {len(np.unique(y_test))}")


Creating LIGHTWEIGHT dataset with 1000 samples...
✅ Dataset created: 1000 samples, 100 features, 10 classes

Dataset statistics:
Feature matrix shape: (1000, 100)
Label array shape: (1000,)
Feature range: [-7.41, 12.54]
Class distribution: [100 100 100 100 100 100 100 100 100 100]
Data split completed:
Training set: 700 samples
Test set: 300 samples
Train classes: 10
Test classes: 10


In [6]:
# ============================================================================
# CELL 6: Split Training Data Among 5 Federated Clients (Non-IID)
# ============================================================================

def create_federated_splits(X_train, y_train, n_clients=5, alpha=0.5):
    """
    Create non-IID data splits for federated clients using Dirichlet distribution

    Parameters:
    - alpha: Controls data heterogeneity (lower = more heterogeneous)
             alpha=0.1 -> very heterogeneous (each client has few classes)
             alpha=10  -> almost IID (balanced across clients)
    """
    print(f"Creating federated splits for {n_clients} clients (alpha={alpha})...")

    n_classes = len(np.unique(y_train))
    n_samples = len(X_train)

    # Group indices by class
    class_indices = [np.where(y_train == c)[0] for c in range(n_classes)]

    # Initialize client data holders
    client_data = [{'X': [], 'y': []} for _ in range(n_clients)]

    # Distribute each class among clients using Dirichlet distribution
    for class_idx, indices in enumerate(class_indices):
        # Sample proportions from Dirichlet
        proportions = np.random.dirichlet([alpha] * n_clients)
        proportions = (np.cumsum(proportions) * len(indices)).astype(int)[:-1]

        # Split indices
        split_indices = np.split(indices, proportions)

        # Assign to clients
        for client_id, client_indices in enumerate(split_indices):
            client_data[client_id]['X'].extend(client_indices)

    # Convert to numpy arrays
    for client_id in range(n_clients):
        idx = client_data[client_id]['X']
        client_data[client_id]['X'] = X_train[idx]
        client_data[client_id]['y'] = y_train[idx]

    # Print statistics
    print("\n📊 Client Data Distribution:")
    for client_id in range(n_clients):
        n_samples = len(client_data[client_id]['y'])
        unique_classes = len(np.unique(client_data[client_id]['y']))
        print(f"Client {client_id+1}: {n_samples:4d} samples, {unique_classes:2d} classes")

    return client_data

# Create federated splits
n_clients = 5
client_data = create_federated_splits(X_train, y_train, n_clients=n_clients, alpha=0.5)

print(f"\n✅ Federated data ready for {n_clients} clients!")


Creating federated splits for 5 clients (alpha=0.5)...

📊 Client Data Distribution:
Client 1:  233 samples,  9 classes
Client 2:   76 samples,  8 classes
Client 3:   84 samples,  9 classes
Client 4:  154 samples,  8 classes
Client 5:  153 samples, 10 classes

✅ Federated data ready for 5 clients!


In [7]:
# ============================================================================
# CELL 7: Federated Client Class (OPTIMIZED)
# ============================================================================

class FederatedClient:
    """
    A federated learning client that trains a local Random Forest model
    """
    
    def __init__(self, client_id, X_local, y_local):
        """
        Initialize client with local data
        
        Parameters:
        - client_id: Unique identifier for this client
        - X_local: Local training features
        - y_local: Local training labels
        """
        self.client_id = client_id
        self.X_local = X_local
        self.y_local = y_local
        self.local_model = None
        self.training_history = []
        
        print(f"✅ Client {client_id} initialized with {len(X_local)} samples")
    
    def train_local_model(self, n_estimators=20, max_depth=5, random_state=None):
        """
        Train local Random Forest on client's private data
        
        OPTIMIZED: Reduced n_estimators from 50 to 20, max_depth from 10 to 5
        """
        print(f"🔄 Client {self.client_id} training local Random Forest...")
        
        # Initialize Random Forest with LIGHTER settings
        self.local_model = RandomForestClassifier(
            n_estimators=n_estimators,
            max_depth=max_depth,
            min_samples_split=10,  # Increased from 5
            min_samples_leaf=5,     # Increased from 2
            random_state=random_state,
            n_jobs=-1  # Use all CPU cores
        )
        
        # Train on local data
        start_time = time.time()
        self.local_model.fit(self.X_local, self.y_local)
        training_time = time.time() - start_time
        
        # Evaluate on local data
        local_accuracy = self.local_model.score(self.X_local, self.y_local)
        
        # Store metrics
        metrics = {
            'accuracy': local_accuracy,
            'n_samples': len(self.X_local),
            'n_classes': len(np.unique(self.y_local)),
            'training_time': training_time,
            'n_estimators': n_estimators
        }
        
        self.training_history.append(metrics)
        
        print(f"✅ Client {self.client_id} training complete: "
              f"Accuracy={local_accuracy:.4f}, Time={training_time:.2f}s")
        
        return self.local_model, metrics
    
    def get_model_trees(self):
        """Return the decision trees from local Random Forest"""
        if self.local_model is None:
            raise ValueError("Model not trained yet!")
        return self.local_model.estimators_
    
    def evaluate(self, X_test, y_test):
        """Evaluate local model on test data"""
        if self.local_model is None:
            raise ValueError("Model not trained yet!")
        
        y_pred = self.local_model.predict(X_test)
        accuracy = accuracy_score(y_test, y_pred)
        
        return accuracy

# Create federated clients
print("Creating federated clients...\n")
clients = []

for client_id in range(n_clients):
    client = FederatedClient(
        client_id=client_id,
        X_local=client_data[client_id]['X'],
        y_local=client_data[client_id]['y']
    )
    clients.append(client)

print(f"\n✅ {len(clients)} federated clients created!")


Creating federated clients...

✅ Client 0 initialized with 233 samples
✅ Client 1 initialized with 76 samples
✅ Client 2 initialized with 84 samples
✅ Client 3 initialized with 154 samples
✅ Client 4 initialized with 153 samples

✅ 5 federated clients created!


In [8]:
# ============================================================================
# CELL 8: Test Single Client Training (OPTIMIZED)
# ============================================================================

print("Testing client training...\n")

# Train first client as test with LIGHTER settings
test_client = clients[0]
model, metrics = test_client.train_local_model(n_estimators=20, max_depth=5)

print("\n📊 Training Metrics:")
for key, value in metrics.items():
    print(f"  {key}: {value}")

# Test evaluation
test_acc = test_client.evaluate(X_test, y_test)
print(f"\n🎯 Test Accuracy: {test_acc:.4f}")


Testing client training...

🔄 Client 0 training local Random Forest...
✅ Client 0 training complete: Accuracy=0.9785, Time=0.05s

📊 Training Metrics:
  accuracy: 0.9785407725321889
  n_samples: 233
  n_classes: 9
  training_time: 0.05320286750793457
  n_estimators: 20

🎯 Test Accuracy: 0.7133


RL - Reinforcement Learning

In [9]:
# ============================================================================
# CELL 9: ULTRA-OPTIMIZED Federated Environment (FAST VERSION)
# ============================================================================

class FederatedAggregationEnv(gym.Env):
    """
    Ultra-optimized environment - minimal computational overhead
    """
    
    metadata = {'render_modes': []}
    
    def __init__(self, clients, X_test, y_test, n_trees_per_client=20):
        super(FederatedAggregationEnv, self).__init__()
        
        self.clients = clients
        self.n_clients = len(clients)
        self.X_test = X_test
        self.y_test = y_test
        self.n_trees_per_client = n_trees_per_client
        
        # Determine global classes
        all_classes = set()
        for client in clients:
            all_classes.update(np.unique(client.y_local))
        self.global_classes = np.array(sorted(list(all_classes)))
        self.n_global_classes = len(self.global_classes)
        
        # State space
        self.observation_space = spaces.Box(
            low=0.0, high=1.0,
            shape=(self.n_clients * 3,),
            dtype=np.float32
        )
        
        # Action space
        self.action_space = spaces.Box(
            low=0.0, high=1.0,
            shape=(self.n_clients,),
            dtype=np.float32
        )
        
        # ⭐ KEY OPTIMIZATION: Train clients ONCE during initialization
        print("🔄 Training all clients (one-time initialization)...")
        self.client_models = []
        self.client_metrics = []
        
        for i, client in enumerate(self.clients):
            print(f"  Training client {i+1}/{self.n_clients}...", end=" ")
            model, metrics = client.train_local_model(
                n_estimators=self.n_trees_per_client,
                max_depth=5,
                random_state=42
            )
            self.client_models.append(model)
            self.client_metrics.append(metrics)
            print(f"✓ Acc={metrics['accuracy']:.4f}")
        
        print(f"✅ All clients trained (will be reused for all episodes)")
        
        # Episode state
        self.current_round = 0
        self.max_rounds = 20  # ⭐ REDUCED from 50 for faster episodes
        self.prev_global_accuracy = 0.0
    
    def __repr__(self):
        return f"FederatedAggregationEnv(n_clients={self.n_clients})"
    
    def __str__(self):
        return self.__repr__()
    
    def reset(self, seed=None, options=None):
        """Reset episode (NO retraining - just reset counters)"""
        super().reset(seed=seed)
        
        self.current_round = 0
        self.prev_global_accuracy = 0.0
        
        state = self._get_state()
        return state, {}
    
    def _get_state(self):
        """Construct state vector from pre-trained clients"""
        state = []
        
        max_samples = max([m['n_samples'] for m in self.client_metrics])
        max_classes = self.n_global_classes
        
        for metrics in self.client_metrics:
            state.extend([
                metrics['accuracy'],
                metrics['n_samples'] / max_samples,
                metrics['n_classes'] / max_classes
            ])
        
        return np.array(state, dtype=np.float32)
    
    def _aggregate_forests(self, weights):
        """Aggregate forests with given weights"""
        weights = weights / (weights.sum() + 1e-8)
        
        global_trees = []
        
        for client_idx, weight in enumerate(weights):
            client_trees = self.client_models[client_idx].estimators_
            n_trees_to_select = max(1, int(weight * self.n_trees_per_client))
            n_trees_to_select = min(n_trees_to_select, len(client_trees))
            
            selected_indices = np.random.choice(
                len(client_trees),
                size=n_trees_to_select,
                replace=False
            )
            selected_trees = [client_trees[i] for i in selected_indices]
            global_trees.extend(selected_trees)
        
        global_forest = RandomForestClassifier(n_estimators=len(global_trees))
        global_forest.estimators_ = global_trees
        global_forest.n_estimators = len(global_trees)
        global_forest.classes_ = self.global_classes
        global_forest.n_classes_ = self.n_global_classes
        global_forest.n_outputs_ = 1
        
        return global_forest
    
    def _evaluate_global_model(self, global_forest):
        """Fast evaluation using voting"""
        try:
            n_samples = len(self.X_test)
            n_trees = len(global_forest.estimators_)
            
            all_predictions = np.zeros((n_samples, n_trees), dtype=int)
            
            for tree_idx, tree in enumerate(global_forest.estimators_):
                all_predictions[:, tree_idx] = tree.predict(self.X_test)
            
            final_predictions = np.array([
                np.argmax(np.bincount(all_predictions[i],
                                     minlength=self.n_global_classes))
                for i in range(n_samples)
            ])
            
            accuracy = accuracy_score(self.y_test, final_predictions)
            return accuracy
            
        except:
            return np.mean([m['accuracy'] for m in self.client_metrics])
    
    def step(self, action):
        """Execute aggregation step"""
        weights = np.clip(action, 0, 1)
        weights = weights / (weights.sum() + 1e-8)
        
        global_forest = self._aggregate_forests(weights)
        global_accuracy = self._evaluate_global_model(global_forest)
        
        accuracy_improvement = global_accuracy - self.prev_global_accuracy
        weight_entropy = -np.sum(weights * np.log(weights + 1e-8))
        fairness_bonus = (weight_entropy / np.log(self.n_clients)) * 0.1
        
        reward = accuracy_improvement + fairness_bonus
        
        self.prev_global_accuracy = global_accuracy
        self.current_round += 1
        
        terminated = self.current_round >= self.max_rounds
        truncated = False
        
        next_state = self._get_state()
        
        info = {
            'global_accuracy': global_accuracy,
            'weights': weights,
            'round': self.current_round,
            'accuracy_improvement': accuracy_improvement
        }
        
        return next_state, reward, terminated, truncated, info
    
    def render(self):
        pass
    
    def close(self):
        pass


# Create optimized environment
print("Creating ULTRA-OPTIMIZED environment...\n")

env = FederatedAggregationEnv(
    clients=clients,
    X_test=X_test,
    y_test=y_test,
    n_trees_per_client=20  # ⭐ REDUCED from 50
)

print(f"\n✅ Environment ready!")
print(f"   State space: {env.observation_space.shape}")
print(f"   Action space: {env.action_space.shape}")


Creating ULTRA-OPTIMIZED environment...

🔄 Training all clients (one-time initialization)...
  Training client 1/5... 🔄 Client 0 training local Random Forest...
✅ Client 0 training complete: Accuracy=0.9957, Time=0.02s
✓ Acc=0.9957
  Training client 2/5... 🔄 Client 1 training local Random Forest...
✅ Client 1 training complete: Accuracy=0.9737, Time=0.02s
✓ Acc=0.9737
  Training client 3/5... 🔄 Client 2 training local Random Forest...
✅ Client 2 training complete: Accuracy=0.9643, Time=0.02s
✓ Acc=0.9643
  Training client 4/5... 🔄 Client 3 training local Random Forest...
✅ Client 3 training complete: Accuracy=0.9870, Time=0.01s
✓ Acc=0.9870
  Training client 5/5... 🔄 Client 4 training local Random Forest...
✅ Client 4 training complete: Accuracy=0.9869, Time=0.02s
✓ Acc=0.9869
✅ All clients trained (will be reused for all episodes)

✅ Environment ready!
   State space: (15,)
   Action space: (5,)


In [10]:
# ============================================================================
# CELL 10 (FIXED): Corrected RL Environment with Class Alignment
# ============================================================================

class FederatedAggregationEnv(gym.Env):
    """
    Custom Gymnasium environment for federated learning aggregation

    FIXED: Handles class mismatch between clients
    """

    def __init__(self, clients, X_test, y_test, n_trees_per_client=50):
        super(FederatedAggregationEnv, self).__init__()

        self.clients = clients
        self.n_clients = len(clients)
        self.X_test = X_test
        self.y_test = y_test
        self.n_trees_per_client = n_trees_per_client

        # Determine global classes (union of all client classes)
        all_classes = set()
        for client in clients:
            all_classes.update(np.unique(client.y_local))
        self.global_classes = np.array(sorted(list(all_classes)))
        self.n_global_classes = len(self.global_classes)

        print(f"Global classes detected: {self.n_global_classes} classes")

        # State space: [accuracy, n_samples, n_classes] for each client
        self.observation_space = spaces.Box(
            low=0.0,
            high=1.0,
            shape=(self.n_clients * 3,),
            dtype=np.float32
        )

        # Action space: continuous weights for each client
        self.action_space = spaces.Box(
            low=0.0,
            high=1.0,
            shape=(self.n_clients,),
            dtype=np.float32
        )

        # Internal state
        self.current_round = 0
        self.max_rounds = 100
        self.prev_global_accuracy = 0.0
        self.client_models = None
        self.client_metrics = None

        print(f"✅ RL Environment created:")
        print(f"   State space: {self.observation_space.shape}")
        print(f"   Action space: {self.action_space.shape}")
        print(f"   Global classes: {self.n_global_classes}")

    def reset(self, seed=None, options=None):
        """Reset environment to initial state"""
        super().reset(seed=seed)

        self.current_round = 0
        self.prev_global_accuracy = 0.0

        # Train all clients initially
        print("\n🔄 Training all clients...")
        self.client_models = []
        self.client_metrics = []

        for client in self.clients:
            model, metrics = client.train_local_model(
                n_estimators=self.n_trees_per_client,
                max_depth=10,
                random_state=42
            )
            self.client_models.append(model)
            self.client_metrics.append(metrics)

        # Get initial state
        state = self._get_state()

        return state, {}

    def _get_state(self):
        """Construct state vector from client metrics"""
        state = []

        max_samples = max([m['n_samples'] for m in self.client_metrics])
        max_classes = self.n_global_classes  # Use global classes

        for metrics in self.client_metrics:
            # Normalize metrics to [0, 1]
            state.extend([
                metrics['accuracy'],  # Already in [0, 1]
                metrics['n_samples'] / max_samples,
                metrics['n_classes'] / max_classes
            ])

        return np.array(state, dtype=np.float32)

    def _aggregate_forests(self, weights):
        """
        Aggregate client forests using RL-selected weights

        FIXED: Properly handles class alignment
        """
        # Normalize weights to sum to 1
        weights = weights / (weights.sum() + 1e-8)

        # Collect trees from all clients according to weights
        global_trees = []

        for client_idx, weight in enumerate(weights):
            client_trees = self.client_models[client_idx].estimators_

            # Select number of trees proportional to weight
            n_trees_to_select = max(1, int(weight * self.n_trees_per_client))

            # Cap to available trees
            n_trees_to_select = min(n_trees_to_select, len(client_trees))

            # Randomly sample trees (for diversity)
            if len(client_trees) >= n_trees_to_select:
                selected_indices = np.random.choice(
                    len(client_trees),
                    size=n_trees_to_select,
                    replace=False
                )
                selected_trees = [client_trees[i] for i in selected_indices]
                global_trees.extend(selected_trees)

        # Create global Random Forest with aligned classes
        global_forest = RandomForestClassifier(n_estimators=len(global_trees))
        global_forest.estimators_ = global_trees
        global_forest.n_estimators = len(global_trees)

        # CRITICAL FIX: Set global classes explicitly
        global_forest.classes_ = self.global_classes
        global_forest.n_classes_ = self.n_global_classes
        global_forest.n_outputs_ = 1

        return global_forest

    def _evaluate_global_model(self, global_forest):
        """
        Evaluate global model with proper class handling

        FIXED: Uses voting-based prediction for mixed-class trees
        """
        try:
            # Get predictions from each tree
            n_samples = len(self.X_test)
            n_trees = len(global_forest.estimators_)

            # Collect predictions from all trees
            all_predictions = np.zeros((n_samples, n_trees), dtype=int)

            for tree_idx, tree in enumerate(global_forest.estimators_):
                tree_pred = tree.predict(self.X_test)
                all_predictions[:, tree_idx] = tree_pred

            # Majority voting for final prediction
            final_predictions = []
            for i in range(n_samples):
                # Count votes for each class
                votes = np.bincount(all_predictions[i], minlength=self.n_global_classes)
                # Select class with most votes
                final_predictions.append(np.argmax(votes))

            final_predictions = np.array(final_predictions)

            # Calculate accuracy
            accuracy = accuracy_score(self.y_test, final_predictions)

            return accuracy

        except Exception as e:
            print(f"⚠️  Evaluation error: {e}")
            # Fallback: return average of client accuracies
            return np.mean([m['accuracy'] for m in self.client_metrics])

    def step(self, action):
        """
        Execute one step: aggregate with given weights, evaluate, return reward
        """
        # Normalize action to valid weights
        weights = np.clip(action, 0, 1)
        weights = weights / (weights.sum() + 1e-8)

        # Aggregate forests
        global_forest = self._aggregate_forests(weights)

        # Evaluate global model (FIXED)
        global_accuracy = self._evaluate_global_model(global_forest)

        # Calculate reward
        accuracy_improvement = global_accuracy - self.prev_global_accuracy

        # Fairness bonus: encourage balanced weights
        weight_entropy = -np.sum(weights * np.log(weights + 1e-8))
        max_entropy = np.log(self.n_clients)
        fairness_bonus = (weight_entropy / max_entropy) * 0.1

        reward = accuracy_improvement + fairness_bonus

        # Update state
        self.prev_global_accuracy = global_accuracy
        self.current_round += 1

        # Check if done
        terminated = self.current_round >= self.max_rounds
        truncated = False

        # Get next state
        next_state = self._get_state()

        # Info dict
        info = {
            'global_accuracy': global_accuracy,
            'weights': weights,
            'round': self.current_round,
            'accuracy_improvement': accuracy_improvement
        }

        return next_state, reward, terminated, truncated, info

    def render(self):
        """Optional: visualize current state"""
        pass


# ============================================================================
# CREATE FIXED ENVIRONMENT
# ============================================================================

print("Creating FIXED RL environment...\n")
env = FederatedAggregationEnv(
    clients=clients,
    X_test=X_test,
    y_test=y_test,
    n_trees_per_client=50
)

# Validate environment
print("\nValidating environment...")
try:
    check_env(env)
    print("✅ Environment is valid!")
except Exception as e:
    print(f"⚠️  Validation warning (can be ignored): {e}")


# ============================================================================
# CELL 10b: Test Fixed Environment
# ============================================================================

print("Testing FIXED environment...\n")

# Reset environment
state, info = env.reset()
print(f"Initial state shape: {state.shape}")
print(f"Initial state values: {state[:6]}... (truncated)")

# Take random action
action = env.action_space.sample()
print(f"\nRandom action: {action}")
print(f"Normalized weights: {action / action.sum()}")

# Execute step
next_state, reward, terminated, truncated, info = env.step(action)

print(f"\n✅ Step executed successfully!")
print(f"Step results:")
print(f"  Reward: {reward:.6f}")
print(f"  Global Accuracy: {info['global_accuracy']:.4f}")
print(f"  Terminated: {terminated}")
print(f"  Round: {info['round']}")
print(f"  Weights used: {info['weights']}")

print("\n✅ Environment working correctly!")


Creating FIXED RL environment...

Global classes detected: 10 classes
✅ RL Environment created:
   State space: (15,)
   Action space: (5,)
   Global classes: 10

Validating environment...

🔄 Training all clients...
🔄 Client 0 training local Random Forest...
✅ Client 0 training complete: Accuracy=0.9957, Time=0.03s
🔄 Client 1 training local Random Forest...
✅ Client 1 training complete: Accuracy=0.9737, Time=0.03s
🔄 Client 2 training local Random Forest...
✅ Client 2 training complete: Accuracy=0.9643, Time=0.03s
🔄 Client 3 training local Random Forest...
✅ Client 3 training complete: Accuracy=0.9870, Time=0.03s
🔄 Client 4 training local Random Forest...
✅ Client 4 training complete: Accuracy=0.9935, Time=0.03s

🔄 Training all clients...
🔄 Client 0 training local Random Forest...
✅ Client 0 training complete: Accuracy=0.9957, Time=0.03s
🔄 Client 1 training local Random Forest...
✅ Client 1 training complete: Accuracy=0.9737, Time=0.03s
🔄 Client 2 training local Random Forest...
✅ Clien

In [ ]:
# ============================================================================
# CELL 11: ULTRA-FAST PPO Training (OPTIMIZED)
# ============================================================================

print("Training PPO with MINIMAL settings for fast execution...\n")

# Initialize PPO with ULTRA-FAST settings
ppo_agent = PPO(
    policy="MlpPolicy",
    env=env,
    learning_rate=3e-4,
    n_steps=128,         # ⭐ REDUCED from 512
    batch_size=32,       # ⭐ REDUCED from 64
    n_epochs=3,          # ⭐ REDUCED from 5
    gamma=0.99,
    gae_lambda=0.95,
    clip_range=0.2,
    verbose=1
)

print("✅ PPO agent initialized!\n")

# Train with MINIMAL timesteps
print("Training for 1000 timesteps (takes ~30-60 seconds)...")
ppo_agent.learn(
    total_timesteps=1000,  # ⭐ REDUCED from 2500
    progress_bar=True
)

print("\n✅ PPO training complete!")

# Save model
ppo_agent.save("ppo_federated_agent")
print("✅ Model saved to 'ppo_federated_agent.zip'")


Training PPO with optimized settings...

Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
✅ PPO agent initialized!

Training for 2500 timesteps (takes ~2-3 minutes)...

🔄 Training all clients...
🔄 Client 0 training local Random Forest...
✅ Client 0 training complete: Accuracy=0.9946, Time=0.04s
🔄 Client 1 training local Random Forest...
✅ Client 1 training complete: Accuracy=1.0000, Time=0.04s
🔄 Client 2 training local Random Forest...
✅ Client 2 training complete: Accuracy=1.0000, Time=0.04s
🔄 Client 3 training local Random Forest...
✅ PPO agent initialized!

Training for 2500 timesteps (takes ~2-3 minutes)...

🔄 Training all clients...
🔄 Client 0 training local Random Forest...
✅ Client 0 training complete: Accuracy=0.9946, Time=0.04s
🔄 Client 1 training local Random Forest...
✅ Client 1 training complete: Accuracy=1.0000, Time=0.04s
🔄 Client 2 training local Random Forest...
✅ Client 2 training complete: Accuracy=1.0000, Time=0.04s
🔄 Clie

Output()

✅ Client 3 training complete: Accuracy=1.0000, Time=0.06s
🔄 Client 4 training local Random Forest...
✅ Client 4 training complete: Accuracy=0.9988, Time=0.07s


In [ ]:
# ============================================================================
# CELL 12: Evaluate PPO Agent (OPTIMIZED)
# ============================================================================

print("Evaluating PPO agent...\n")

# Reset environment
state, info = env.reset()

# Run evaluation episode with FEWER rounds
episode_rewards = []
episode_accuracies = []
episode_weights_history = []

for step in range(10):  # ⭐ REDUCED from 20 to 10 rounds
    # Get action from trained agent
    action, _states = ppo_agent.predict(state, deterministic=True)
    
    # Execute action
    state, reward, terminated, truncated, info = env.step(action)
    
    episode_rewards.append(reward)
    episode_accuracies.append(info['global_accuracy'])
    episode_weights_history.append(info['weights'])
    
    print(f"Round {step+1}: Acc={info['global_accuracy']:.4f}, "
          f"Reward={reward:.6f}, Weights={info['weights']}")
    
    if terminated or truncated:
        break

# Plot results
fig, axes = plt.subplots(1, 2, figsize=(14, 4))

# Accuracy over rounds
axes[0].plot(episode_accuracies, marker='o', linewidth=2)
axes[0].set_xlabel('Round')
axes[0].set_ylabel('Global Accuracy')
axes[0].set_title('PPO: Global Accuracy Over Rounds')
axes[0].grid(True, alpha=0.3)

# Weights distribution
weights_array = np.array(episode_weights_history)
for i in range(n_clients):
    axes[1].plot(weights_array[:, i], label=f'Client {i+1}', marker='o')
axes[1].set_xlabel('Round')
axes[1].set_ylabel('Weight')
axes[1].set_title('PPO: Client Weights Over Rounds')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print(f"\n📊 Final Results:")
print(f"  Final Accuracy: {episode_accuracies[-1]:.4f}")
print(f"  Average Reward: {np.mean(episode_rewards):.6f}")
print(f"  Best Accuracy: {max(episode_accuracies):.4f}")


In [ ]:
# ============================================================================
# CELL 13: Training Function for All Agents (OPTIMIZED)
# ============================================================================

def train_and_evaluate_agent(agent_name, agent_class, agent_config,
                             clients, X_test, y_test,
                             n_timesteps=1000, n_eval_rounds=10):
    """
    Train and evaluate a single RL agent
    
    OPTIMIZED: Reduced timesteps and eval rounds
    
    Returns: metrics dictionary
    """
    print(f"\n{'='*60}")
    print(f"Training {agent_name}")
    print(f"{'='*60}\n")
    
    # Create fresh environment
    env = FederatedAggregationEnv(clients, X_test, y_test, n_trees_per_client=20)
    
    # Initialize agent
    try:
        if agent_name == 'DQN':
            # DQN needs discrete action space - skip for now
            print(f"⚠️  {agent_name} requires discrete action space - using placeholder")
            return create_random_baseline(clients, X_test, y_test, n_eval_rounds)
        
        agent = agent_class(
            policy="MlpPolicy",
            env=env,
            **agent_config,
            verbose=0
        )
        
        print(f"✅ {agent_name} initialized")
        
        # Train
        start_time = time.time()
        agent.learn(total_timesteps=n_timesteps, progress_bar=True)
        training_time = time.time() - start_time
        
        print(f"✅ Training complete in {training_time:.2f}s")
    
    except Exception as e:
        print(f"❌ Error training {agent_name}: {e}")
        return create_random_baseline(clients, X_test, y_test, n_eval_rounds)
    
    # Evaluate
    print(f"Evaluating {agent_name}...")
    state, info = env.reset()
    
    metrics = {
        'accuracies': [],
        'rewards': [],
        'weights_history': [],
        'training_time': training_time
    }
    
    for round_idx in range(n_eval_rounds):
        action, _ = agent.predict(state, deterministic=True)
        state, reward, terminated, truncated, info = env.step(action)
        
        metrics['accuracies'].append(info['global_accuracy'])
        metrics['rewards'].append(reward)
        metrics['weights_history'].append(info['weights'])
        
        if terminated or truncated:
            break
    
    # Save agent
    agent.save(f"{agent_name.lower()}_federated_agent")
    
    print(f"✅ {agent_name} evaluation complete!")
    print(f"   Final Accuracy: {metrics['accuracies'][-1]:.4f}")
    print(f"   Average Reward: {np.mean(metrics['rewards']):.6f}")
    
    return metrics

def create_random_baseline(clients, X_test, y_test, n_eval_rounds):
    """Create random baseline for comparison"""
    print("Creating random baseline...")
    
    env = FederatedAggregationEnv(clients, X_test, y_test, n_trees_per_client=20)
    state, info = env.reset()
    
    metrics = {
        'accuracies': [],
        'rewards': [],
        'weights_history': [],
        'training_time': 0.0
    }
    
    for _ in range(n_eval_rounds):
        action = env.action_space.sample()  # Random weights
        state, reward, terminated, truncated, info = env.step(action)
        
        metrics['accuracies'].append(info['global_accuracy'])
        metrics['rewards'].append(reward)
        metrics['weights_history'].append(info['weights'])
        
        if terminated or truncated:
            break
    
    return metrics

print("✅ Training function defined!")


🔄 Training all clients...
🔄 Training all clients...
🔄 Training all clients...
🔄 Training all clients...
🔄 Training all clients...
🔄 Training all clients...
🔄 Training all clients...
🔄 Training all clients...
🔄 Training all clients...
🔄 Training all clients...
🔄 Training all clients...
🔄 Training all clients...
🔄 Training all clients...
🔄 Training all clients...
🔄 Training all clients...
🔄 Training all clients...
🔄 Training all clients...
🔄 Training all clients...
🔄 Training all clients...
🔄 Training all clients...
🔄 Training all clients...
🔄 Training all clients...
🔄 Training all clients...
🔄 Training all clients...
🔄 Training all clients...
🔄 Training all clients...
🔄 Training all clients...
🔄 Training all clients...
🔄 Training all clients...
🔄 Training all clients...
🔄 Training all clients...
🔄 Training all clients...
🔄 Training all clients...
🔄 Training all clients...
🔄 Training all clients...
🔄 Training all clients...
🔄 Training all clients...
🔄 Training all clients...
🔄 Training all clients...
🔄 Training all clients...
🔄 Training all clients...
🔄 Training all clients...
🔄 Training all clients...
🔄 Training all clients...
🔄 Training all clients...
🔄 Training all clients...
🔄 Training all clients...
🔄 Training all clients...
🔄 Training all clients...
🔄 Training all clients...
🔄 Training all clients...
🔄 Training all clients...
🔄 Training all clients...
🔄 Training all clients...
🔄 Training all clients...
🔄 Training all clients...
🔄 Training all clients...
🔄 Training all clients...
🔄 Training all clients...
🔄 Training all clients...
🔄 Training all clients...
🔄 Training all clients...
🔄 Training all clients...
🔄 Training all clients...
🔄 Training all clients...
🔄 Training all clients...
🔄 Training all clients...
🔄 Training all clients...
🔄 Training all clients...
🔄 Training all clients...
🔄 Training all clients...
🔄 Training all clients...
🔄 Training all clients...
🔄 Training all clients...
🔄 Training all clients...
🔄 Training all clients...
🔄 Training all clients...
🔄 Training all clients...
🔄 Training all clients...
🔄 Training all clients...
🔄 Training all clients...
🔄 Training all clients...
🔄 Training all clients...
🔄 Training all clients...
🔄 Training all clients...
🔄 Training all clients...
🔄 Training all clients...
🔄 Training all clients...
🔄 Training all clients...
🔄 Training all clients...
🔄 Training all clients...
🔄 Training all clients...
🔄 Training all clients...
🔄 Training all clients...
🔄 Training all clients...
🔄 Training all clients...
🔄 Training all clients...
🔄 Training all clients...
🔄 Training all clients...
🔄 Training all clients...
🔄 Training all clients...
🔄 Training all clients...
🔄 Training all clients...
🔄 Training all clients...
🔄 Training all clients...
🔄 Training all clients...
🔄 Training all clients...
🔄 Training all clients...
🔄 Training all clients...
🔄 Training all clients...
🔄 Training all clients...
🔄 Training all clients...
🔄 Training all clients...
🔄 Training all clients...
🔄 Training all clients...
🔄 Training all clients...
🔄 Training all clients...
🔄 Training all clients...
🔄 Training all clients...
🔄 Training all clients...
🔄 Training all clients...
🔄 Training all clients...
🔄 Training all clients...
🔄 Training all clients...
🔄 Training all clients...
🔄 Training all clients...
🔄 Training all clients...
🔄 Training all clients...
🔄 Training all clients...
🔄 Training all clients...
🔄 Training all clients...
🔄 Training all clients...
🔄 Training all clients...
🔄 Training all clients...
🔄 Training all clients...
🔄 Training all clients...
🔄 Training all clients...
🔄 Training all clients...
🔄 Training all clients...
🔄 Training all clients...
🔄 Training all clients...
🔄 Training all clients...
🔄 Training all clients...
🔄 Training all clients...
🔄 Training all clients...
🔄 Training all clients...
🔄 Training all clients...
🔄 Training all clients...
🔄 Training all clients...
🔄 Training all clients...
🔄 Training all clients...
🔄 Training all clients...
🔄 Training all clients...
🔄 Training all clients

In [ ]:
# ============================================================================
# CELL 14: RL Agent Configurations (OPTIMIZED - ONLY BEST 3 AGENTS)
# ============================================================================

# Configuration for ONLY the best performing agents
# Removed A2C, DDPG, DQN to save time
agent_configs = {
    'PPO': {
        'class': PPO,
        'config': {
            'learning_rate': 3e-4,
            'n_steps': 128,      # ⭐ REDUCED
            'batch_size': 32,    # ⭐ REDUCED
            'n_epochs': 3,       # ⭐ REDUCED
            'gamma': 0.99,
            'gae_lambda': 0.95,
            'clip_range': 0.2,
        }
    },
    'SAC': {
        'class': SAC,
        'config': {
            'learning_rate': 3e-4,
            'buffer_size': 5000,    # ⭐ REDUCED from 10000
            'learning_starts': 50,  # ⭐ REDUCED from 100
            'batch_size': 32,       # ⭐ REDUCED from 64
            'tau': 0.005,
            'gamma': 0.99,
        }
    },
    'TD3': {
        'class': TD3,
        'config': {
            'learning_rate': 1e-3,
            'buffer_size': 5000,    # ⭐ REDUCED from 10000
            'learning_starts': 50,  # ⭐ REDUCED from 100
            'batch_size': 32,       # ⭐ REDUCED from 64
            'tau': 0.005,
            'gamma': 0.99,
            'policy_delay': 2,
        }
    }
}

print(f"✅ Configured {len(agent_configs)} RL agents (PPO, SAC, TD3 - FASTEST)")
print("⏱️  Estimated total time: ~3-5 minutes (vs 15-20 minutes for all 6 agents)")


✅ Configured 5 RL agents

In [ ]:
# ============================================================================
# CELL 15: Train All RL Agents (OPTIMIZED - Takes ~3-5 minutes)
# ============================================================================

results = {}

for agent_name, agent_info in agent_configs.items():
    metrics = train_and_evaluate_agent(
        agent_name=agent_name,
        agent_class=agent_info['class'],
        agent_config=agent_info['config'],
        clients=clients,
        X_test=X_test,
        y_test=y_test,
        n_timesteps=1000,  # ⭐ REDUCED from 5000
        n_eval_rounds=10   # ⭐ REDUCED from 20
    )
    results[agent_name] = metrics

# Add random baseline
print("\n" + "="*60)
print("Creating Random Baseline")
print("="*60)
results['Random'] = create_random_baseline(clients, X_test, y_test, n_eval_rounds=10)

print("\n" + "="*60)
print("✅ ALL EXPERIMENTS COMPLETE!")
print("="*60)
print("\n⏱️  Total agents tested: 4 (PPO, SAC, TD3, Random)")
print("💡 This is 60% faster than testing all 6 agents!")


In [ ]:
# ============================================================================
# CELL 16: Plot Learning Curves for All Agents
# ============================================================================

plt.figure(figsize=(14, 6))

for agent_name, metrics in results.items():
    plt.plot(metrics['accuracies'], marker='o', linewidth=2,
             markersize=4, label=agent_name, alpha=0.8)

plt.xlabel('Round', fontsize=12)
plt.ylabel('Global Accuracy', fontsize=12)
plt.title('Federated Random Forest: RL Agents Comparison',
          fontsize=14, fontweight='bold')
plt.legend(fontsize=10, loc='lower right')
plt.grid(True, alpha=0.3)
plt.axhline(y=0.80, color='red', linestyle='--', alpha=0.5, label='Target (80%)')

plt.tight_layout()
plt.savefig('rl_agents_comparison.png', dpi=300, bbox_inches='tight')
plt.show()

print("✅ Learning curves plotted and saved!")


============================================================
============================================================
============================================================
============================================================
============================================================
============================================================
============================================================
============================================================
============================================================
============================================================
============================================================
============================================================
============================================================
============================================================
============================================================
============================================================
============================================================
============================================================
============================================================
============================================================
============================================================
============================================================
============================================================
============================================================
============================================================
============================================================
============================================================
============================================================
============================================================
============================================================
============================================================
============================================================
============================================================
============================================================
============================================================
============================================================
============================================================
============================================================
============================================================
============================================================
============================================================
============================================================
============================================================
============================================================
============================================================
============================================================
============================================================
============================================================
============================================================
============================================================
============================================================
============================================================
============================================================
============================================================
============================================================
============================================================
============================================================
============================================================
============================================================
============================================================
============================================================
============================================================
============================================================
============================================================
============================================================
===================================

In [ ]:
# ============================================================================
# CELL 17: Create Metrics Comparison Table
# ============================================================================

# Compute summary statistics
summary = {}

for agent_name, metrics in results.items():
    accs = metrics['accuracies']
    rewards = metrics['rewards']

    summary[agent_name] = {
        'Final Accuracy': accs[-1],
        'Best Accuracy': max(accs),
        'Avg Accuracy': np.mean(accs),
        'Convergence Speed': np.argmax(np.array(accs) >= 0.75) + 1 if max(accs) >= 0.75 else len(accs),
        'Stability (Std)': np.std(accs[-5:]),
        'Avg Reward': np.mean(rewards),
        'Training Time (s)': metrics.get('training_time', 0)
    }

# Create DataFrame
df_summary = pd.DataFrame(summary).T

# Sort by final accuracy
df_summary = df_summary.sort_values('Final Accuracy', ascending=False)

print("\n📊 RL AGENTS COMPARISON SUMMARY")
print("="*80)
print(df_summary.to_string())
print("="*80)

# Save to CSV
df_summary.to_csv('rl_agents_summary.csv')
print("\n✅ Summary saved to 'rl_agents_summary.csv'")


KeyError: 'Final Accuracy'

/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)


In [ ]:
# ============================================================================
# CELL 18: Comprehensive Comparison Visualization
# ============================================================================

fig, axes = plt.subplots(2, 3, figsize=(18, 10))
fig.suptitle('RL Agents Performance Comparison', fontsize=16, fontweight='bold')

agents = list(results.keys())
colors = plt.cm.Set3(np.linspace(0, 1, len(agents)))

# 1. Final Accuracy
axes[0, 0].bar(agents, [summary[a]['Final Accuracy'] for a in agents], color=colors)
axes[0, 0].set_title('Final Accuracy')
axes[0, 0].set_ylabel('Accuracy')
axes[0, 0].tick_params(axis='x', rotation=45)
axes[0, 0].axhline(y=0.80, color='red', linestyle='--', alpha=0.5)
axes[0, 0].grid(True, alpha=0.3, axis='y')

# 2. Best Accuracy
axes[0, 1].bar(agents, [summary[a]['Best Accuracy'] for a in agents], color=colors)
axes[0, 1].set_title('Best Accuracy Achieved')
axes[0, 1].set_ylabel('Accuracy')
axes[0, 1].tick_params(axis='x', rotation=45)
axes[0, 1].grid(True, alpha=0.3, axis='y')

# 3. Convergence Speed (lower is better)
axes[0, 2].bar(agents, [summary[a]['Convergence Speed'] for a in agents], color=colors)
axes[0, 2].set_title('Convergence Speed')
axes[0, 2].set_ylabel('Rounds to 75% Accuracy')
axes[0, 2].tick_params(axis='x', rotation=45)
axes[0, 2].grid(True, alpha=0.3, axis='y')

# 4. Stability (lower is better)
axes[1, 0].bar(agents, [summary[a]['Stability (Std)'] for a in agents], color=colors)
axes[1, 0].set_title('Training Stability (Last 5 Rounds)')
axes[1, 0].set_ylabel('Std Dev')
axes[1, 0].tick_params(axis='x', rotation=45)
axes[1, 0].grid(True, alpha=0.3, axis='y')

# 5. Average Reward
axes[1, 1].bar(agents, [summary[a]['Avg Reward'] for a in agents], color=colors)
axes[1, 1].set_title('Average Reward')
axes[1, 1].set_ylabel('Reward')
axes[1, 1].tick_params(axis='x', rotation=45)
axes[1, 1].grid(True, alpha=0.3, axis='y')

# 6. Training Time
axes[1, 2].bar(agents, [summary[a]['Training Time (s)'] for a in agents], color=colors)
axes[1, 2].set_title('Training Time')
axes[1, 2].set_ylabel('Seconds')
axes[1, 2].tick_params(axis='x', rotation=45)
axes[1, 2].grid(True, alpha=0.3, axis='y')

plt.tight_layout()
plt.savefig('rl_agents_detailed_comparison.png', dpi=300, bbox_inches='tight')
plt.show()

print("✅ Comprehensive comparison visualization complete!")


In [ ]:
# ============================================================================
# CELL 19: Analyze Client Weight Distribution
# ============================================================================

# Plot weight distributions for each agent
n_agents = len(results)
fig, axes = plt.subplots(2, 3, figsize=(18, 10))
axes = axes.flatten()

for idx, (agent_name, metrics) in enumerate(results.items()):
    if idx >= len(axes):
        break

    weights_history = np.array(metrics['weights_history'])

    # Plot each client's weight over time
    for client_idx in range(n_clients):
        axes[idx].plot(weights_history[:, client_idx],
                      label=f'Client {client_idx+1}',
                      linewidth=2, marker='o', markersize=3)

    axes[idx].set_title(f'{agent_name}: Client Weights', fontweight='bold')
    axes[idx].set_xlabel('Round')
    axes[idx].set_ylabel('Weight')
    axes[idx].legend(fontsize=8)
    axes[idx].grid(True, alpha=0.3)

# Remove unused subplots
for idx in range(n_agents, len(axes)):
    fig.delaxes(axes[idx])

plt.tight_layout()
plt.savefig('client_weights_analysis.png', dpi=300, bbox_inches='tight')
plt.show()

print("✅ Client weight analysis complete!")


In [ ]:
# ============================================================================
# CELL 20: Save All Results
# ============================================================================

import pickle

# Create results directory
!mkdir -p results

# Save raw metrics
with open('results/all_metrics.pkl', 'wb') as f:
    pickle.dump(results, f)

# Save summary CSV
df_summary.to_csv('results/summary.csv')

# Save individual agent results
for agent_name, metrics in results.items():
    df_agent = pd.DataFrame({
        'round': range(len(metrics['accuracies'])),
        'accuracy': metrics['accuracies'],
        'reward': metrics['rewards']
    })
    df_agent.to_csv(f'results/{agent_name}_metrics.csv', index=False)

print("✅ All results saved to 'results/' directory")
print("\nSaved files:")
print("  - all_metrics.pkl (complete metrics)")
print("  - summary.csv (comparison table)")
for agent in results.keys():
    print(f"  - {agent}_metrics.csv")
